In [3]:
# using ast.literal_eval()
import sys
import ast
import os
import time
import pandas as pd
import pickle
import glob
import numpy as np
import statistics
from scipy.stats import mannwhitneyu
import math

In [4]:
with open("STRING_paths.dictionary", 'rb') as config_dictionary_file:
 
    # Step 3
    paths = pickle.load(config_dictionary_file)

In [5]:
# create list
file2 = "common_nodes_hPPIN_STRING_PANCAN.txt"
combined_list = []
with open(file2) as IN:
    for line in IN:
        combined_list.append(line.rstrip().split("\t")[0])

In [6]:
def compute_sp(list1, list2, paths):
    total_sp = []
    for i in range(len(list1)):
        for j in range(len(list2)):
            if(list1[i] in paths.keys()):
                shortest_paths = paths[list1[i]]
                if(list2[j] in shortest_paths.keys()):
                    total_sp.append(shortest_paths[list2[j]])
    return({"distribution": total_sp,
            "mean": statistics.mean(total_sp),
            "median": statistics.median(total_sp)})

In [7]:
# Read Cancersea Data
folder_path = "/data/timonaj/cancer_as_wound/ppi_analysis/hallmark"

filenames = os.listdir(folder_path)

hallmark = {}

for filename in filenames:
    file_path = f"{folder_path}/{filename}"
    hallmark[filename[:-4]] = list()

    with open(file_path) as IN:
        for line in IN:
            hallmark[filename[:-4]].append(line.rstrip())

In [8]:
hallmark["COSMIC"] = list()
with open("/data/timonaj/cancer_as_wound/ppi_analysis/cosmic_hallmarks.txt") as IN:
    for line in IN:
        hallmark["COSMIC"].append(line.rstrip().split("\t")[0])

In [9]:
# read in HP sets
folder_path = "/data/timonaj/cancer_as_wound/ppi_analysis/HP_genesets/"

filenames = os.listdir(folder_path)

hp_genesets = {}

for filename in filenames:
    file_path = f"{folder_path}/{filename}"
    hp_genesets[filename[:-4]] = list()

    with open(file_path) as IN:
        for line in IN:
            hp_genesets[filename[:-4]].append(line.rstrip().split("\t")[0])

In [11]:
# read in HP matched controls set
folder_path = "/data/timonaj/cancer_as_wound/ppi_analysis/HP_genesets_matched_controls/"

filenames = glob.glob("/data/timonaj/cancer_as_wound/ppi_analysis/HP_genesets_matched_controls/STRING*")
hp_genesets_matched = {}

for filename in filenames:
    name = filename.split("STRING_")[1][:-4]
    hp_genesets_matched[name] = list()

    with open(filename) as IN:
        for line in IN:
            hp_genesets_matched[name].append(line.rstrip().split("\t")[0])

In [12]:
a_mean = np.zeros(shape=(len(hp_genesets), len(hallmark)))
a_median = np.zeros(shape=(len(hp_genesets), len(hallmark)))
a_dist = pd.DataFrame(index = range(len(hallmark)),
                      columns = range(len(hp_genesets)))

In [13]:
i=0
for key1,value1 in hp_genesets.items():
    j=0
    for key2,value2 in hallmark.items():
        current_info = compute_sp(value1, value2, paths)
        a_mean[i][j] = current_info["mean"]
        a_median[i][j] = current_info["median"]
        a_dist[i][j] = current_info["distribution"]
        j+=1
    i+=1

In [14]:
pd.DataFrame(a_median)

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
1,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
2,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
3,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
4,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
5,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
6,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
7,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
8,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0


In [15]:
pd.DataFrame(a_mean)

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,1.907395,1.853133,1.887515,1.843961,1.897219,1.886926,1.769910,1.915982,1.881036,1.898314,...,1.892195,1.905166,1.933268,1.837556,1.895260,1.812651,1.897216,1.811413,1.872622,1.876206
1,1.937940,1.917893,1.899584,1.912709,1.921625,1.911085,1.893266,1.924200,1.895889,1.903728,...,1.914108,1.912484,1.939483,1.924626,1.911339,1.867765,1.924124,1.897567,1.889656,1.905815
2,1.927605,1.873749,1.882328,1.878527,1.918527,1.907663,1.842904,1.928138,1.879933,1.919983,...,1.909835,1.926451,1.938545,1.888906,1.912406,1.850447,1.912755,1.830647,1.905310,1.893855
3,1.888774,1.796385,1.906091,1.807271,1.902788,1.871558,1.730323,1.935890,1.884270,1.888180,...,1.877927,1.904184,1.943335,1.807910,1.914792,1.756612,1.877688,1.740723,1.871845,1.839376
4,1.900777,1.817503,1.848710,1.821325,1.866739,1.850082,1.762657,1.883898,1.827757,1.871871,...,1.858065,1.883531,1.914721,1.837454,1.856993,1.740016,1.864087,1.744507,1.851090,1.808712
5,1.922530,1.880525,1.886338,1.876991,1.907280,1.902320,1.839383,1.917573,1.886357,1.910163,...,1.904523,1.915062,1.937356,1.886526,1.899023,1.843168,1.911195,1.853655,1.890107,1.897597
6,1.917038,1.882646,1.903801,1.876766,1.904875,1.895219,1.831676,1.920694,1.893638,1.901867,...,1.894471,1.899265,1.936358,1.881342,1.896865,1.830956,1.905345,1.845243,1.881343,1.887634
7,1.903325,1.883515,1.844374,1.857483,1.866292,1.872786,1.798984,1.889746,1.831651,1.905274,...,1.882700,1.901798,1.903278,1.863643,1.842998,1.766631,1.894262,1.844371,1.856067,1.846122
8,1.933527,1.924658,1.919368,1.915974,1.902547,1.911505,1.904802,1.896564,1.918995,1.900596,...,1.913936,1.903763,1.937058,1.932789,1.887554,1.874733,1.925683,1.915092,1.879349,1.922818


In [16]:
pd.DataFrame(a_mean, columns = hallmark.keys(), index = hp_genesets.keys()).to_csv("STRING_meanSP_HP_Phenotypes_hallmark.csv")

In [17]:
a_matched_mean = np.zeros(shape=(len(hp_genesets_matched), len(hallmark)))
a_matched_median = np.zeros(shape=(len(hp_genesets_matched), len(hallmark)))
a_matched_dist = pd.DataFrame(index = range(len(hallmark)),
                              columns = range(len(hp_genesets_matched)))

In [18]:
i=0
for key1,value1 in hp_genesets_matched.items():
    j=0
    for key2,value2 in hallmark.items():
        current_info = compute_sp(value1, value2, paths)
        a_matched_mean[i][j] = current_info["mean"]
        a_matched_median[i][j] = current_info["median"]
        a_matched_dist[i][j] = current_info["distribution"]
        j+=1
    i+=1

In [19]:
pd.DataFrame(a_matched_median)

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
1,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
2,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
3,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
4,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
5,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
6,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
7,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
8,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0


In [20]:
pd.DataFrame(a_matched_mean)

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,2.948838,2.941503,2.861646,2.936267,2.938528,2.930357,2.878468,2.937375,2.866253,2.969330,...,2.921424,2.942126,2.921917,2.943580,2.908315,2.823257,2.935934,2.932359,2.917592,2.891891
1,2.953481,2.947624,2.864793,2.946055,2.945108,2.936834,2.883856,2.944358,2.870883,2.973342,...,2.927212,2.947161,2.928062,2.950213,2.914011,2.833206,2.942864,2.938542,2.924713,2.904050
2,2.947016,2.936912,2.859653,2.931635,2.936952,2.928165,2.869239,2.937161,2.863821,2.966819,...,2.918999,2.940097,2.921282,2.936537,2.907103,2.817909,2.933001,2.926559,2.916406,2.889993
3,2.962751,2.962984,2.872176,2.967342,2.956951,2.949602,2.903928,2.955161,2.881197,2.982439,...,2.939138,2.957337,2.938409,2.969401,2.923764,2.855778,2.957287,2.955567,2.937271,2.925789
4,2.948430,2.940143,2.861081,2.934949,2.938288,2.929830,2.875556,2.937582,2.865566,2.968692,...,2.920818,2.941650,2.921879,2.941440,2.908237,2.821673,2.935171,2.930590,2.917400,2.891486
5,2.949690,2.940854,2.861602,2.937076,2.940465,2.931636,2.874200,2.940426,2.866498,2.969451,...,2.922275,2.942979,2.924127,2.941441,2.910272,2.823551,2.936873,2.930851,2.919833,2.895551
6,2.945824,2.935982,2.859072,2.929538,2.935013,2.926502,2.869672,2.934775,2.862869,2.965986,...,2.917604,2.938834,2.919373,2.936009,2.905296,2.815970,2.931369,2.925912,2.914301,2.886505
7,2.975341,2.977401,2.879944,2.992667,2.975691,2.967275,2.912073,2.976271,2.893007,2.992562,...,2.954572,2.970904,2.956401,2.983258,2.940333,2.880869,2.975669,2.969083,2.957728,2.960516
8,2.941908,2.930399,2.856265,2.920823,2.929582,2.920936,2.864207,2.929127,2.858751,2.962551,...,2.912618,2.934553,2.914217,2.929773,2.900765,2.806980,2.925342,2.920128,2.908279,2.876006


In [21]:
pd.DataFrame(a_matched_mean, columns = hallmark.keys(), index = hp_genesets_matched.keys()).to_csv("STRING_meanSP_HPmatched_Phenotypes_hallmark.csv")

In [22]:
mannwhitneyu_p = np.zeros(shape=(len(hp_genesets), len(hallmark)))
test_dist = np.zeros(shape=(len(hp_genesets), len(hallmark)))
test_matched_dist = np.zeros(shape=(len(hp_genesets), len(hallmark)))

In [23]:
LogFC = np.zeros(shape=(len(hp_genesets), len(hallmark)))
for i in range(len(hp_genesets)):
    for j in range(len(hallmark)):
        actual_dist = a_dist[i][j]
        control_dist = a_matched_dist[i][j]
        res = mannwhitneyu(actual_dist, control_dist, alternative="less")
        mannwhitneyu_p[i][j] = res.pvalue
        test_dist[i][j] = statistics.mean(actual_dist)
        test_matched_dist[i][j] = statistics.mean(control_dist)
        LogFC[i][j] = math.log((test_dist[i][j]/test_matched_dist[i][j]))

In [24]:
hallmark.keys()

dict_keys(['PROTEIN_SECRETION', 'COAGULATION', 'E2F_TARGETS', 'COMPLEMENT', 'UNFOLDED_PROTEIN_RESPONSE', 'UV_RESPONSE_UP', 'IL6_JAK_STAT3_SIGNALING', 'MYC_TARGETS_V2', 'G2M_CHECKPOINT', 'XENOBIOTIC_METABOLISM', 'ESTROGEN_RESPONSE_EARLY', 'HEME_METABOLISM', 'WNT_BETA_CATENIN_SIGNALING', 'APOPTOSIS', 'KRAS_SIGNALING_UP', 'APICAL_SURFACE', 'NOTCH_SIGNALING', 'TNFA_SIGNALING_VIA_NFKB', 'GLYCOLYSIS', 'APICAL_JUNCTION', 'IL2_STAT5_SIGNALING', 'P53_PATHWAY', 'PANCREAS_BETA_CELLS', 'BILE_ACID_METABOLISM', 'INTERFERON_ALPHA_RESPONSE', 'HEDGEHOG_SIGNALING', 'ALLOGRAFT_REJECTION', 'MITOTIC_SPINDLE', 'TGF_BETA_SIGNALING', 'UV_RESPONSE_DN', 'KRAS_SIGNALING_DN', 'PEROXISOME', 'FATTY_ACID_METABOLISM', 'EPITHELIAL_MESENCHYMAL_TRANSITION', 'SPERMATOGENESIS', 'MTORC1_SIGNALING', 'CHOLESTEROL_HOMEOSTASIS', 'OXIDATIVE_PHOSPHORYLATION', 'MYOGENESIS', 'INTERFERON_GAMMA_RESPONSE', 'HYPOXIA', 'ANDROGEN_RESPONSE', 'ADIPOGENESIS', 'DNA_REPAIR', 'INFLAMMATORY_RESPONSE', 'MYC_TARGETS_V1', 'PI3K_AKT_MTOR_SIGNALING

In [25]:
pvals = pd.DataFrame(mannwhitneyu_p, columns = hallmark.keys(), index = hp_genesets.keys())

In [26]:
pvals

,PROTEIN_SECRETION,COAGULATION,E2F_TARGETS,COMPLEMENT,UNFOLDED_PROTEIN_RESPONSE,UV_RESPONSE_UP,IL6_JAK_STAT3_SIGNALING,MYC_TARGETS_V2,G2M_CHECKPOINT,XENOBIOTIC_METABOLISM,...,ANDROGEN_RESPONSE,ADIPOGENESIS,DNA_REPAIR,INFLAMMATORY_RESPONSE,MYC_TARGETS_V1,PI3K_AKT_MTOR_SIGNALING,ESTROGEN_RESPONSE_LATE,ANGIOGENESIS,REACTIVE_OXYGEN_SPECIES_PATHWAY,COSMIC
downregulated.wound,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
upregulated.stress,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
downregulated.regen,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
msigdb.wound,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
msigdb.regen,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
upregulated.regen,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
upregulated.wound,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
msigdb.stress,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
downregulated.stress,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
pvals.to_csv("STRING_matched_pvals_hallmark.csv")

In [28]:
LFC = pd.DataFrame(LogFC, columns = hallmark.keys(), index = hp_genesets.keys())

In [29]:
LFC

,PROTEIN_SECRETION,COAGULATION,E2F_TARGETS,COMPLEMENT,UNFOLDED_PROTEIN_RESPONSE,UV_RESPONSE_UP,IL6_JAK_STAT3_SIGNALING,MYC_TARGETS_V2,G2M_CHECKPOINT,XENOBIOTIC_METABOLISM,...,ANDROGEN_RESPONSE,ADIPOGENESIS,DNA_REPAIR,INFLAMMATORY_RESPONSE,MYC_TARGETS_V1,PI3K_AKT_MTOR_SIGNALING,ESTROGEN_RESPONSE_LATE,ANGIOGENESIS,REACTIVE_OXYGEN_SPECIES_PATHWAY,COSMIC
downregulated.wound,-0.435673,-0.462043,-0.416136,-0.465223,-0.437520,-0.440175,-0.486329,-0.427286,-0.421183,-0.447370,...,-0.434334,-0.434563,-0.413028,-0.471190,-0.428218,-0.443101,-0.436638,-0.481700,-0.443419,-0.432659
upregulated.stress,-0.421359,-0.429772,-0.410861,-0.431947,-0.426975,-0.429661,-0.420825,-0.425381,-0.414932,-0.445872,...,-0.424799,-0.432439,-0.411920,-0.427146,-0.421726,-0.416667,-0.424913,-0.437341,-0.436801,-0.421196
downregulated.regen,-0.424515,-0.449418,-0.418191,-0.445072,-0.425814,-0.428497,-0.442704,-0.420889,-0.420921,-0.435174,...,-0.424224,-0.422763,-0.410085,-0.441233,-0.418795,-0.420568,-0.427482,-0.469158,-0.425707,-0.422639
msigdb.wound,-0.450190,-0.500420,-0.410016,-0.495849,-0.440839,-0.454899,-0.517756,-0.422986,-0.424665,-0.457128,...,-0.447948,-0.440235,-0.413463,-0.496189,-0.423263,-0.485957,-0.454231,-0.529390,-0.450557,-0.464138
msigdb.regen,-0.439010,-0.480994,-0.436711,-0.477126,-0.453634,-0.459714,-0.489424,-0.444244,-0.449676,-0.461183,...,-0.452328,-0.445822,-0.422655,-0.470518,-0.448589,-0.483436,-0.453994,-0.518732,-0.454918,-0.469155
upregulated.regen,-0.428058,-0.447149,-0.416744,-0.447744,-0.432889,-0.432486,-0.446344,-0.427494,-0.418444,-0.441188,...,-0.428131,-0.429672,-0.411672,-0.444162,-0.426907,-0.426510,-0.429617,-0.458133,-0.434893,-0.422587
upregulated.wound,-0.429607,-0.444364,-0.406645,-0.445294,-0.432295,-0.434473,-0.448967,-0.423944,-0.413324,-0.444374,...,-0.431823,-0.436546,-0.410560,-0.445066,-0.426333,-0.430469,-0.430806,-0.460996,-0.437644,-0.424722
msigdb.stress,-0.446756,-0.457911,-0.445631,-0.476943,-0.466523,-0.460217,-0.481643,-0.454229,-0.457079,-0.451504,...,-0.450647,-0.446067,-0.440395,-0.470483,-0.467129,-0.489018,-0.451640,-0.476115,-0.465962,-0.472276
downregulated.stress,-0.419713,-0.420390,-0.397519,-0.421639,-0.431666,-0.424013,-0.407914,-0.434661,-0.398584,-0.443883,...,-0.419890,-0.432723,-0.408431,-0.415961,-0.429692,-0.403643,-0.418130,-0.421862,-0.436636,-0.402611


In [30]:
LFC.to_csv("STRING_matched_LFC_hallmark.csv")

In [31]:
pd.DataFrame(test_dist)

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,1.907395,1.853133,1.887515,1.843961,1.897219,1.886926,1.769910,1.915982,1.881036,1.898314,...,1.892195,1.905166,1.933268,1.837556,1.895260,1.812651,1.897216,1.811413,1.872622,1.876206
1,1.937940,1.917893,1.899584,1.912709,1.921625,1.911085,1.893266,1.924200,1.895889,1.903728,...,1.914108,1.912484,1.939483,1.924626,1.911339,1.867765,1.924124,1.897567,1.889656,1.905815
2,1.927605,1.873749,1.882328,1.878527,1.918527,1.907663,1.842904,1.928138,1.879933,1.919983,...,1.909835,1.926451,1.938545,1.888906,1.912406,1.850447,1.912755,1.830647,1.905310,1.893855
3,1.888774,1.796385,1.906091,1.807271,1.902788,1.871558,1.730323,1.935890,1.884270,1.888180,...,1.877927,1.904184,1.943335,1.807910,1.914792,1.756612,1.877688,1.740723,1.871845,1.839376
4,1.900777,1.817503,1.848710,1.821325,1.866739,1.850082,1.762657,1.883898,1.827757,1.871871,...,1.858065,1.883531,1.914721,1.837454,1.856993,1.740016,1.864087,1.744507,1.851090,1.808712
5,1.922530,1.880525,1.886338,1.876991,1.907280,1.902320,1.839383,1.917573,1.886357,1.910163,...,1.904523,1.915062,1.937356,1.886526,1.899023,1.843168,1.911195,1.853655,1.890107,1.897597
6,1.917038,1.882646,1.903801,1.876766,1.904875,1.895219,1.831676,1.920694,1.893638,1.901867,...,1.894471,1.899265,1.936358,1.881342,1.896865,1.830956,1.905345,1.845243,1.881343,1.887634
7,1.903325,1.883515,1.844374,1.857483,1.866292,1.872786,1.798984,1.889746,1.831651,1.905274,...,1.882700,1.901798,1.903278,1.863643,1.842998,1.766631,1.894262,1.844371,1.856067,1.846122
8,1.933527,1.924658,1.919368,1.915974,1.902547,1.911505,1.904802,1.896564,1.918995,1.900596,...,1.913936,1.903763,1.937058,1.932789,1.887554,1.874733,1.925683,1.915092,1.879349,1.922818


In [32]:
pd.DataFrame(test_matched_dist)

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,2.948838,2.941503,2.861646,2.936267,2.938528,2.930357,2.878468,2.937375,2.866253,2.969330,...,2.921424,2.942126,2.921917,2.943580,2.908315,2.823257,2.935934,2.932359,2.917592,2.891891
1,2.953481,2.947624,2.864793,2.946055,2.945108,2.936834,2.883856,2.944358,2.870883,2.973342,...,2.927212,2.947161,2.928062,2.950213,2.914011,2.833206,2.942864,2.938542,2.924713,2.904050
2,2.947016,2.936912,2.859653,2.931635,2.936952,2.928165,2.869239,2.937161,2.863821,2.966819,...,2.918999,2.940097,2.921282,2.936537,2.907103,2.817909,2.933001,2.926559,2.916406,2.889993
3,2.962751,2.962984,2.872176,2.967342,2.956951,2.949602,2.903928,2.955161,2.881197,2.982439,...,2.939138,2.957337,2.938409,2.969401,2.923764,2.855778,2.957287,2.955567,2.937271,2.925789
4,2.948430,2.940143,2.861081,2.934949,2.938288,2.929830,2.875556,2.937582,2.865566,2.968692,...,2.920818,2.941650,2.921879,2.941440,2.908237,2.821673,2.935171,2.930590,2.917400,2.891486
5,2.949690,2.940854,2.861602,2.937076,2.940465,2.931636,2.874200,2.940426,2.866498,2.969451,...,2.922275,2.942979,2.924127,2.941441,2.910272,2.823551,2.936873,2.930851,2.919833,2.895551
6,2.945824,2.935982,2.859072,2.929538,2.935013,2.926502,2.869672,2.934775,2.862869,2.965986,...,2.917604,2.938834,2.919373,2.936009,2.905296,2.815970,2.931369,2.925912,2.914301,2.886505
7,2.975341,2.977401,2.879944,2.992667,2.975691,2.967275,2.912073,2.976271,2.893007,2.992562,...,2.954572,2.970904,2.956401,2.983258,2.940333,2.880869,2.975669,2.969083,2.957728,2.960516
8,2.941908,2.930399,2.856265,2.920823,2.929582,2.920936,2.864207,2.929127,2.858751,2.962551,...,2.912618,2.934553,2.914217,2.929773,2.900765,2.806980,2.925342,2.920128,2.908279,2.876006
